**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

See [Configure Service Account](https://kubernetes.io/docs/tasks/configure-pod-container/configure-service-account/)

In [1]:
kubectl get serviceaccount default -o yaml

apiVersion: v1
kind: ServiceAccount
metadata:
  creationTimestamp: "2021-04-27T14:48:01Z"
  name: default
  namespace: default
  resourceVersion: "396"
  uid: af7a65d4-6b90-4e12-9008-fcd0dc9cf568
secrets:
- name: default-token-dwhww


In [2]:
kubectl patch serviceaccount default  --patch "automountServiceAccountToken: false"

serviceaccount/default patched


In [3]:
kubectl get serviceaccount default -o yaml

apiVersion: v1
automountServiceAccountToken: false
kind: ServiceAccount
metadata:
  creationTimestamp: "2021-04-27T14:48:01Z"
  name: default
  namespace: default
  resourceVersion: "29721"
  uid: af7a65d4-6b90-4e12-9008-fcd0dc9cf568
secrets:
- name: default-token-dwhww


# ☠️ **this can be overwritten** ☠️



In [4]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-with-token-again
  labels:
    app: nginx-with-token-again
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx-with-token-again
  template:
    metadata:
      labels:
        app: nginx-with-token-again
    spec:
      automountServiceAccountToken: true
      containers:
      - name: nginx
        image: nginxinc/nginx-unprivileged
        ports:
        - containerPort: 8080
EOF

deployment.apps/nginx-with-token-again created


In [8]:
kubectl get pods
POD=$(kubectl get pods -o jsonpath='{.items[0].metadata.name}' )
echo $POD

NAME                                      READY   STATUS    RESTARTS   AGE
nginx-strict-5cdc48cf55-27r9c             1/1     Running   0          4m28s
nginx-strict-5cdc48cf55-k65vn             1/1     Running   0          4m28s
nginx-strict-5cdc48cf55-rwkt5             1/1     Running   0          4m28s
nginx-with-token-again-66868cff8b-dncj2   1/1     Running   0          42s
nginx-strict-5cdc48cf55-27r9c


In [9]:
kubectl exec  nginx-with-token-again-66868cff8b-dncj2 -- df

Filesystem             1K-blocks      Used Available Use% Mounted on
overlay                308587328 143851296 148991008  50% /
tmpfs                      65536         0     65536   0% /dev
tmpfs                   32666292         0  32666292   0% /sys/fs/cgroup
/dev/mapper/system-var 308587328 143851296 148991008  50% /etc/hosts
shm                        65536         0     65536   0% /dev/shm
tmpfs                   32666292        12  32666280   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs                   32666292         0  32666292   0% /proc/acpi
tmpfs                   32666292         0  32666292   0% /proc/scsi
tmpfs                   32666292         0  32666292   0% /sys/firmware


In [10]:
kubectl get pods  -o yaml | grep -i token

    generateName: nginx-with-token-again-66868cff8b-
      app: nginx-with-token-again
    name: nginx-with-token-again-66868cff8b-dncj2
      name: nginx-with-token-again-66868cff8b
    automountServiceAccountToken: true
        name: default-token-dwhww
    - name: default-token-dwhww
        secretName: default-token-dwhww


In [11]:
kubectl get pods -A -o jsonpath='{.items[*].spec.automountServiceAccountToken}'

true